In [16]:
import dask
import dask_image.imread
import dask.distributed
import skimage.io as io

import numpy as np
import os

from sklearn.svm import SVC
from sklearn import metrics
from Emotions_Detection.Models.features import lpq, lpq_plus, PHOG_Algorithm

In [17]:
# Constants and Vaiables
IMG_TRAIN_PATH = "../Assets/data/train"
IMG_TEST_PATH = "../Assets/data/test"
HAPPY_PATH = "../Assets/data/train/happy/"

train_folders = os.listdir(IMG_TRAIN_PATH)  # Emotions Folders
test_folders = os.listdir(IMG_TEST_PATH)  # Emotions Folders

# Intialize dash board for DASK
# client = dask.distributed.Client()  # Create a local cluster  
cluster = dask.distributed.LocalCluster(n_workers=12, threads_per_worker=2)
client = dask.distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 24,Total memory: 15.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54441,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 15.91 GiB
Comm: tcp://127.0.0.1:54522,Total threads: 2
Dashboard: http://127.0.0.1:54524/status,Memory: 1.33 GiB
Nanny: tcp://127.0.0.1:54447,


### **Auxilary Functions:**

In [18]:
# Auxiary Function Calls
@dask.delayed
def read_img_parallel(img_path):
    return io.imread(img_path, as_gray=True)

# Reading A Directory of images folders
def read_data(path: str):
    x_img = []
    y_img = []
    tasks = []
    folders = os.listdir(path)
    for folder in folders:
        folder_path = os.path.join(path, folder)
        pattern = os.path.join(folder_path, 'im*.png')
        imgs = dask_image.imread.imread(pattern)
        tasks.append(imgs)
        # Set Labels While Initializing
        labels = [folder for _ in range(len(imgs))]
        y_img.extend(labels)
    
    print("Implementing Tasks...")
    for task in tasks:
        x_img.extend(task.compute())
        print("Done...")
    return x_img, y_img

def get_phog(images):
    phog = []
    for img in images:
        phog_desc = dask.delayed(PHOG_Algorithm)(img, 8, 3)
        phog.append(phog_desc)

    print('Starting PHOG Set...')
    phog = dask.compute(phog)[0]
    print('Done PHOG Set...')
    return phog

def get_lpq(images):
    lpq_train = []
    for img in images:
        lpq_desc = dask.delayed(lpq)(img)
        lpq_train.append(lpq_desc)

    print('Starting LPQ Set...')
    lpq_train = dask.compute(lpq_train)[0]
    print('Done LPQ Set...')
    return lpq_train
        